In [1]:
import pandas as pd
import numpy as np

In [9]:
orders = pd.read_csv("data/orders.csv")
order_products = pd.read_csv("data/order_products.csv")
joint = pd.merge(orders, order_products.drop_duplicates(subset=['order_id']), how="left", on="order_id")
# ref. https://stackoverflow.com/questions/22720739/pandas-left-outer-join-results-in-table-larger-than-left-table
joint.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order
0,2539329,1,1,2,8,NaN,196.0,1.0
1,2398795,1,2,3,7,15.0,196.0,1.0
2,473747,1,3,3,12,21.0,196.0,1.0
3,2254736,1,4,4,7,29.0,196.0,1.0
4,431534,1,5,4,15,28.0,196.0,1.0


In [13]:
joint.shape[0]

3421083

In [14]:
orders.shape[0]

3421083

<b>--- Create a new column "eval_set" and fill with NaN <br/>
--- If product_id is null, set eval_set="test".</b>

In [15]:
joint["eval_set"] = np.nan
# set "test" where product_id is null
joint["eval_set"][joint["product_id"].isnull()] = "test"


<ipython-input-15-7fc09d16c575>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joint["eval_set"][joint["product_id"].isnull()] = "test"


<b>--- Check the last order of each user.<br/>
--- When eval_set="test", product_id=NaN, add_to_cart_order=Nan.<br/>
--- Seems fine.</b>

In [16]:
joint.loc[joint.reset_index().groupby(["user_id"])["order_number"].idxmax()]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,eval_set
10,1187899,1,11,4,8,14.0,196.0,1.0,NaN
25,1492625,2,15,1,11,30.0,22963.0,1.0,NaN
38,2774568,3,13,5,15,11.0,NaN,NaN,test
44,329954,4,6,3,12,30.0,NaN,NaN,test
49,2196797,5,5,0,11,6.0,15349.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...
3420933,1716008,206205,4,1,16,10.0,27845.0,1.0,NaN
3421001,1043943,206206,68,0,20,0.0,NaN,NaN,test
3421018,2821651,206207,17,2,13,14.0,NaN,NaN,test
3421068,803273,206208,50,5,11,4.0,NaN,NaN,test


<b>--- In the above dataframe, rows where eval_set = NaN should be assigned eval_set="train".<br/>
--- That is because the last orders of a user with a product_id is train data.<br/>
--- Exact those row indexes and assign.</b>

In [17]:
last_orders = joint.loc[joint.reset_index().groupby(["user_id"])["order_number"].idxmax()]["order_id"]

# row index of last orders
last_orders.index.tolist()
len(last_orders.index.tolist())

206209

In [18]:
# set "train"
joint.iloc[last_orders.index.tolist(),-1] = joint.iloc[last_orders.index.tolist(),-1].fillna("train")

<b>--- Check again the last order of each user.<br/>
--- Now we have "test" and "train".<br/>
--- Seems fine.</b>

In [19]:
joint.loc[joint.reset_index().groupby(["user_id"])["order_number"].idxmax()]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,eval_set
10,1187899,1,11,4,8,14.0,196.0,1.0,train
25,1492625,2,15,1,11,30.0,22963.0,1.0,train
38,2774568,3,13,5,15,11.0,NaN,NaN,test
44,329954,4,6,3,12,30.0,NaN,NaN,test
49,2196797,5,5,0,11,6.0,15349.0,1.0,train
...,...,...,...,...,...,...,...,...,...
3420933,1716008,206205,4,1,16,10.0,27845.0,1.0,train
3421001,1043943,206206,68,0,20,0.0,NaN,NaN,test
3421018,2821651,206207,17,2,13,14.0,NaN,NaN,test
3421068,803273,206208,50,5,11,4.0,NaN,NaN,test


<b>--- Set all other remaining rows as eval_set="prior".<br/></b>

In [20]:
values = {'eval_set': "prior"}
joint.fillna(value=values, inplace=True)

<b>--- Final check.<br/></b>

In [21]:
joint['eval_set'].isnull().sum()

0

In [22]:
test = joint[joint["eval_set"] == "test"]
test

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,eval_set
38,2774568,3,13,5,15,11.0,NaN,NaN,test
44,329954,4,6,3,12,30.0,NaN,NaN,test
53,1528013,6,4,3,16,22.0,NaN,NaN,test
96,1376945,11,8,6,11,8.0,NaN,NaN,test
102,1356845,12,6,1,20,30.0,NaN,NaN,test
...,...,...,...,...,...,...,...,...,...
3420918,2728930,206202,23,2,17,6.0,NaN,NaN,test
3420929,350108,206204,5,4,14,14.0,NaN,NaN,test
3421001,1043943,206206,68,0,20,0.0,NaN,NaN,test
3421018,2821651,206207,17,2,13,14.0,NaN,NaN,test


In [23]:
train = joint[joint["eval_set"] == "train"]
train

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,eval_set
10,1187899,1,11,4,8,14.0,196.0,1.0,train
25,1492625,2,15,1,11,30.0,22963.0,1.0,train
49,2196797,5,5,0,11,6.0,15349.0,1.0,train
74,525192,7,21,2,11,6.0,12053.0,1.0,train
78,880375,8,4,1,14,10.0,15937.0,1.0,train
...,...,...,...,...,...,...,...,...,...
3420838,2585586,206199,20,2,16,30.0,29429.0,1.0,train
3420862,943915,206200,24,6,19,6.0,22828.0,1.0,train
3420924,2371631,206203,6,4,19,30.0,15693.0,1.0,train
3420933,1716008,206205,4,1,16,10.0,27845.0,1.0,train


In [24]:
prior = joint[joint["eval_set"] == "prior"]
prior

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,eval_set
0,2539329,1,1,2,8,NaN,196.0,1.0,prior
1,2398795,1,2,3,7,15.0,196.0,1.0,prior
2,473747,1,3,3,12,21.0,196.0,1.0,prior
3,2254736,1,4,4,7,29.0,196.0,1.0,prior
4,431534,1,5,4,15,28.0,196.0,1.0,prior
...,...,...,...,...,...,...,...,...,...
3421077,2558525,206209,9,4,15,22.0,38167.0,1.0,prior
3421078,2266710,206209,10,5,18,29.0,9405.0,1.0,prior
3421079,1854736,206209,11,4,10,30.0,6846.0,1.0,prior
3421080,626363,206209,12,1,12,18.0,6846.0,1.0,prior


In [25]:
print("rows of prior: ", prior.shape[0])
print("rows of train: ", train.shape[0])
print("rows of test: ", test.shape[0])

rows of prior:  3214874
rows of train:  131209
rows of test:  75000
